## TEXTURE SEGMENTATION


In [47]:
import gc
gc.collect()


1233

In [48]:
import os
import warnings
import numpy as np
from glob import glob
from PIL import Image
from matplotlib import pyplot as plt
from scipy import fft
from scipy.signal import find_peaks
from scipy.ndimage import binary_dilation
import cv2
from skimage.exposure import rescale_intensity

from skimage import (
    io, color, morphology, filters, transform, data
)
from skimage.color import rgb2gray, label2rgb
from skimage.filters.rank import entropy
from skimage.feature import graycoprops, graycomatrix
from skimage.segmentation import find_boundaries, mark_boundaries
from skimage.transform import probabilistic_hough_line
from skimage.draw import line as skimage_line
from skimage.measure import label
from skimage.morphology import remove_small_objects, disk
import warnings
from felzenszwalb_segmentation import segment

plt.rc({'family': 'normal', 'weight': 'normal'})
plt.rcParams['font.size'] = 18
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Ignore DeprecationWarning
warnings.filterwarnings('ignore', category=DeprecationWarning, module='skimage')


## MAIN ALGORITHM

### Function for preprocessing

In [49]:
def preprocessing(img):

    # Scale the image to the 0-255 range and convert to unsigned 8-bit integers
    img = (img * 255).astype(np.uint8)

    sElem = disk(40)
    pageFloat = img
    # Apply a local mean filter using the structuring element and normalize the result
    pageFilt = filters.rank.mean(pageFloat, footprint=sElem) / 255
    img_processed = pageFloat - pageFilt

    # Adjust the contrast of the image
    img_processed = cv2.convertScaleAbs(img_processed, alpha=0.65, beta=0)

    # Apply a median blur to reduce noise
    img_processed = cv2.medianBlur(np.uint8(img_processed), 5)
    # Apply a Gaussian blur for additional smoothing
    blurred_img = cv2.GaussianBlur(img_processed, (11, 11), 0)

    sElem = disk(60)
    pageFloat = blurred_img
    pageFilt = filters.rank.mean(pageFloat, footprint=sElem) / 255
    # Subtract the second local mean-filtered image from the smoothed image
    img_processed = pageFloat - pageFilt

    return img_processed


### Function for frequency filter

In [50]:
def freq_filter(img):
    """
    Perform frequency-based filtering and morphological operations on the input image.

    Parameters:
    img (ndarray): Input grayscale image.

    Returns:
    ndarray: Processed binary image after high-pass filtering and morphological operations.
    """
    # Apply a local mean filter with a disk-shaped structuring element (radius=40)
    sElem = disk(40)
    pageFloat = img
    pageFilt = filters.rank.mean(pageFloat, footprint=sElem) / 255
    img_processed = pageFloat - pageFilt

    # Step 1: Perform Fourier Transform to analyze frequency components
    img_f = fft.fft2(img_processed)
    zero_center_img_fft = fft.fftshift(img_f)  # Center the zero frequency for better manipulation

    # Step 2: Apply high-pass filtering in the frequency domain
    pixels = 160  # Threshold to define the high-frequency region to filter out
    img_size = img.shape
    filt_spect = zero_center_img_fft
    filt_spect[img_size[0] // 2 - pixels:img_size[0] // 2 + pixels,
    img_size[1] // 2 - pixels:img_size[1] // 2 + pixels] = 0 + 0j  # Set low-frequency region to zero

    # Step 3: Perform inverse Fourier Transform to convert back to the spatial domain
    inverse_img = fft.ifft2(filt_spect)
    real_img = np.abs(inverse_img) * 255  # Take the magnitude and scale to 0-255
    real_img_uint8 = real_img.astype(np.uint8)  # Convert to unsigned 8-bit integers

    # Step 5: Binarize the image with a threshold
    val = real_img_uint8 > 12  # Threshold to distinguish foreground and background
    val_uint8 = (val * 255).astype(np.uint8)  # Convert the binary result to 8-bit format

    # Step 6: Perform morphological opening to remove small noise
    kernel = np.ones((1, 1), np.uint8)  # Small kernel size for precise noise removal
    opened_image = cv2.morphologyEx(val_uint8, cv2.MORPH_OPEN, kernel)

    # Step 7: Perform morphological closing to fill small gaps in the foreground
    kernel = np.ones((4, 4), np.uint8)  # Larger kernel to bridge gaps
    closed_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, kernel)

    return closed_image


In [51]:
def freq_filter1(img):
    """
    Apply a low-pass frequency filter to the input image using Fourier Transform.

    Parameters:
    img (ndarray): Input grayscale image.

    Returns:
    ndarray: Reconstructed image after applying the low-pass filter.
    """
    # Scale the image to the range 0-255 and convert to unsigned 8-bit integers
    img = (img * 255).astype(np.uint8)

    # Define the size of the low-frequency region to retain in the filtered spectrum
    pixels = 80

    # Perform 2D Fourier Transform to move to the frequency domain
    img_fft = fft.fft2(img)
    zero_center_img_fft = fft.fftshift(img_fft)  # Shift the zero frequency to the center

    # Get the size of the image
    img_size = img.shape

    # Create an empty array for the filtered spectrum (complex values)
    filt_spect = np.zeros(zero_center_img_fft.shape, dtype=np.complex128)

    # Retain only the low-frequency components within the defined region
    filt_spect[img_size[0] // 2 - pixels:img_size[0] // 2 + pixels,
    img_size[1] // 2 - pixels:img_size[1] // 2 + pixels] = \
        zero_center_img_fft[img_size[0] // 2 - pixels:img_size[0] // 2 + pixels,
        img_size[1] // 2 - pixels:img_size[1] // 2 + pixels]

    # Perform the inverse Fourier Transform to reconstruct the image in the spatial domain
    inverseImg = np.abs(fft.ifft2(filt_spect))

    return inverseImg


### Function for Hough transform

In [52]:
def hough_transform(img):
    """
    Perform probabilistic Hough Transform to detect lines in the input binary image.
    The parameters for line detection are dynamically adjusted based on the ratio of white pixels.

    Parameters:
    img (ndarray): Input binary image.

    Returns:
    ndarray: Binary image with detected lines.
    """
    # Calculate the number of white pixels and their ratio relative to the total pixels
    white_pixel_count = np.sum(img > 0)
    total_pixel_count = img.shape[0] * img.shape[1]
    white_pixel_ratio = 1000 * white_pixel_count / total_pixel_count  # Scale ratio for better parameter adjustment
    #print('\nwhite_pixel_ratio:', white_pixel_ratio)

    # Dynamically compute parameters for Hough Transform based on white_pixel_ratio
    threshold = int(1.22 * white_pixel_ratio + 0.93)  # Minimum votes needed to detect a line
    line_length = int(-1.63 * white_pixel_ratio + 71.76)  # Minimum length of a line segment
    line_gap = int(-3.37 * white_pixel_ratio + 134.96)  # Maximum allowed gap between line segments
    #print(f"threshold: {threshold}, line_length: {line_length}, line_gap: {line_gap}")

    # Apply probabilistic Hough Transform to detect lines
    lines = probabilistic_hough_line(img, threshold=threshold, line_length=line_length, line_gap=line_gap)

    # Initialize a blank image to draw the detected lines
    result = np.zeros_like(img)
    for line in lines:
        p0, p1 = line  # Extract start and end points of each line
        rr, cc = skimage_line(p0[1], p0[0], p1[1], p1[0])  # Generate line coordinates
        result[rr, cc] = 255  # Draw the line in white on the result image

    # Step 1: Morphological opening to remove small noise
    kernel = np.ones((1, 1), np.uint8)
    opened_image = cv2.morphologyEx(result, cv2.MORPH_OPEN, kernel)

    # Step 2: Morphological closing to fill small gaps in detected lines
    kernel = np.ones((4, 4), np.uint8)
    closed_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, kernel)

    # Step 3: Additional opening to refine the result
    kernel = np.ones((1, 1), np.uint8)
    opened_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel)

    return closed_image


### Function for combining image and lines

In [53]:
def combine(blur, lines):
    """
    Combine a grayscale image with a binary line image by overlaying the lines on the grayscale image.

    Parameters:
    blur (ndarray): Grayscale image with values in the range [0, 255] or [0, 1].
    lines (ndarray): Binary image with line information, values in the range [0, 255] or [0, 1].

    Returns:
    ndarray: Grayscale image with lines overlaid in white.
    """
    # Ensure the grayscale image is in the range 0-255
    blur = (blur * 255).astype(np.uint8) if blur.max() <= 1 else blur

    # Ensure the binary line image is in the range 0-255
    lines = (lines * 255).astype(np.uint8) if lines.max() <= 1 else lines

    # Create a copy of the grayscale image to avoid modifying the original
    output_image = blur.copy()

    # Overlay the lines: set pixels to white (255) where the binary line image has 255
    output_image[lines == 255] = 255

    return output_image


### Function for Felzenszwalb Segmentation

In [54]:
def felz_segm(im_contrast):
    """
    Apply Felzenszwalb segmentation to the input grayscale image.

    Parameters:
    im_contrast (ndarray): Grayscale input image.

    Returns:
    ndarray: Segmented image with regions labeled as unique integers.
    """
    # Convert the grayscale image to RGB by stacking the single channel three times
    im_contrast1 = np.stack([im_contrast] * 3, axis=-1)

    #print(im_contrast1.shape)

    # Apply Felzenszwalb segmentation with specified hyperparameters
    segmented_image = segment(im_contrast1, 0, 55, 900)

    return segmented_image.astype(np.uint8)  # Convert the segmented result to uint8 format


### Function for morphology

In [55]:
def postprocessing(img):
    """
    Apply postprocessing to the input image, including grayscale conversion and morphological closing.

    Parameters:
    img (ndarray): Input image, either grayscale or RGB.

    Returns:
    ndarray: Processed image after applying morphological closing.
    """
    # Check if the image is in RGB format and convert it to grayscale if necessary
    if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    # Perform morphological opening to remove small noise

    kernel = np.ones((7, 7), np.uint8)
    opened_image = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

    img_inverted = cv2.bitwise_not(opened_image)
    # Apply the morphological closing operation (dilation followed by erosion)
    #kernel = np.ones((5, 5), np.uint8)
    #closed_image = cv2.morphologyEx(img_inverted, cv2.MORPH_CLOSE, kernel)

    img_normalized = img_inverted.astype(np.float32) / 255.0

    return img_normalized


### Algo function

In [56]:
def main_alg(data_folder_path):

    count = len([f for f in os.listdir(data_folder_path) if f.endswith('.png')])
    if not os.path.exists("output"):
        os.makedirs("output")

    # Main algorithm loop
    for idx in range(1, count+1):
        input_path = f"data/tm{idx}_1_1.png"
        if input_path.endswith('.png'):  # Check if the file has a .png extension (modify if needed)
            image = plt.imread(input_path)
        # Apply preprocessing steps
        blur = preprocessing(image)
        # Apply frequency domain filtering
        blur = freq_filter1(blur)

        # Apply frequency-based texture extraction
        lines = freq_filter(image)
        # Perform Hough Transform to detect lines
        lines = hough_transform(lines)

        # Combine the blurred image and detected lines to generate texture
        texture = combine(blur, lines)

        # Apply segmentation to refine the texture
        texture = felz_segm(texture)

        # Apply morphology
        texture = postprocessing(texture)



        imgx = Image.fromarray((texture * 255).astype(np.uint8))  # Convert to 8-bit image
        output_path = f"output/seg{idx}_1_1.png"

        imgx.save(output_path)  # Save the image to the specified path
        print(f"Image saved: {output_path}")




## Calling algo!

In [57]:
if __name__ == "__main__":

    data_folder_path = 'data'
    #output_folder_path = 'output'
    main_alg(data_folder_path)

/tmp/ipykernel_7800/3010502576.py:44: UserWarning: Possible precision loss converting image of type float32 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  lines = freq_filter(image)


Image saved: output/seg1_1_1.png
Image saved: output/seg2_1_1.png
Image saved: output/seg3_1_1.png
Image saved: output/seg4_1_1.png
Image saved: output/seg5_1_1.png
Image saved: output/seg6_1_1.png
Image saved: output/seg7_1_1.png
Image saved: output/seg8_1_1.png
Image saved: output/seg9_1_1.png
Image saved: output/seg10_1_1.png
Image saved: output/seg11_1_1.png
Image saved: output/seg12_1_1.png
Image saved: output/seg13_1_1.png
Image saved: output/seg14_1_1.png
Image saved: output/seg15_1_1.png
Image saved: output/seg16_1_1.png
Image saved: output/seg17_1_1.png
Image saved: output/seg18_1_1.png
Image saved: output/seg19_1_1.png
Image saved: output/seg20_1_1.png
